In [3]:
// T1 locales (Tier 1 markets)
const t1Locales = [
  { locale: '', name: 'US (English)' },
  { locale: 'de', name: 'Germany' },
  { locale: 'fr', name: 'France' },
  { locale: 'jp', name: 'Japan' },
  { locale: 'uk', name: 'United Kingdom' },
  { locale: 'es', name: 'Spain' },
  { locale: 'it', name: 'Italy' },
  { locale: 'in', name: 'India' },
  { locale: 'br', name: 'Brazil' },
];

// Frictionless PDF pages
const frictionlessPdfPages = [
  'pdf-to-word',
  'pdf-to-jpg',
  'pdf-to-excel',
  'pdf-to-ppt',
  'convert-pdf',
  'word-to-pdf',
  'jpg-to-pdf',
  'png-to-pdf',
  'excel-to-pdf',
  'ppt-to-pdf',
  'ocr-pdf',
  'compress-pdf',
  'pdf-editor',
  'merge-pdf',
  'split-pdf',
  'crop-pdf',
  'delete-pdf-pages',
  'rotate-pdf',
  'rearrange-pdf',
  'extract-pdf-pages',
  'add-pages-to-pdf',
  'add-pdf-page-numbers',
  'sign-pdf',
  'request-signature',
  'password-protect-pdf',
  'ai-chat-pdf',
];

const compressPdfVariations = [
  'compress-pdf-50kb',
  'compress-pdf-100kb',
  'compress-pdf-200kb',
  'compress-pdf-300kb',
  'compress-pdf-500kb',
  'compress-pdf-1mb',
];

console.log(`Testing ${t1Locales.length} T1 locales × ${frictionlessPdfPages.length} frictionless PDF pages = ${t1Locales.length * frictionlessPdfPages.length} total URLs`);
console.log(`Testing IN locale × ${compressPdfVariations.length} compress PDF pages = ${compressPdfVariations.length} total URLs`);


Testing 9 T1 locales × 26 frictionless PDF pages = 234 total URLs
Testing IN locale × 6 compress PDF pages = 6 total URLs


In [4]:
// Function to build URL for a locale and page
function buildUrl(env, locale, page) {
  let baseUrl = 'https://www.adobe.com';
  if (env === 'stage') {
    baseUrl = 'https://www.stage.adobe.com';
  }
  const localePrefix = locale ? `/${locale}` : '';
  return `${baseUrl}${localePrefix}/acrobat/online/${page}.html`;
}

// Function to check if a URL is on EdgeWorker
async function checkEdgeWorker(url) {
  try {
    const startTime = Date.now();
    const response = await fetch(url, {
      method: 'GET',
      headers: {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
      }
    });
    const responseTime = Date.now() - startTime;
    
    const headers = Object.fromEntries(response.headers.entries());
    
    // Check for EdgeWorker indicators
    const edgeWorkerIndicators = {
      hasEwFrictionlessHeader: !!headers['content-security-policy'],
      responseTime,
      status: response.status,
      statusText: response.statusText
    };
    
    return {
      url,
      success: true,
      edgeWorkerIndicators,
      headers,
      isOnEdgeWorker: edgeWorkerIndicators.hasEwFrictionlessHeader // Primary indicator
    };
  } catch (error) {
    return {
      url,
      success: false,
      error: error.message,
      isOnEdgeWorker: false
    };
  }
}

// Test the function with one URL
const testUrl = buildUrl('stage', 'in', 'ocr-pdf');
console.log('Testing URL:', testUrl);
const testResult = await checkEdgeWorker(testUrl);
console.log('Test result:', testResult);


Testing URL: https://www.stage.adobe.com/in/acrobat/online/ocr-pdf.html
Test result: {
  url: "https://www.stage.adobe.com/in/acrobat/online/ocr-pdf.html",
  success: true,
  edgeWorkerIndicators: {
    hasEwFrictionlessHeader: true,
    responseTime: 225,
    status: 200,
    statusText: "OK"
  },
  headers: {
    "access-control-allow-origin": "**/online/**",
    "akamai-cache-status": "Hit from child",
    "akamai-grn": "0.28622c17.1752709900.1d53206d, 0.28622c17.1752710350.1d648de2",
    "content-security-policy": "child-src 'self' blob: ;  connect-src 'self' blob: 14257-chimera-stage.adobeioruntime.net 14257-chimera.adobeioruntime.net *.adobe.com stage.adobeccstatic.com *.adobe.io *.identity-stage.adobe.com *.adobelogin.com *.amazonaws.com *.blob.core.windows.net *.clicktale.net/ *.mktoresp.com tag-logger.demandbase.com accounts.google.com/gsi/ acom-reviews--adobe.hlx.live/reviews-api/ ade0164.d41.co/ adobe.tt.omtrdc.net adobedc.demdex.net/ adobeioruntime.net adobesearch-stage.ado

In [12]:
// Function to add delay between requests to avoid rate limiting
function delay(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

// Main function to check all URLs
async function checkAllUrls(env) {
  const results = [];
  let totalChecked = 0;
  let onEdgeWorker = 0;
  let errors = 0;
  
  console.log('Starting EdgeWorker check for all frictionless PDF pages in T1 locales...');
  console.log('=' .repeat(80));
  
  for (const localeInfo of t1Locales) {
    console.log(`\nChecking ${localeInfo.name} (${localeInfo.locale || 'default'})...`);
    
    const localeResults = [];
    
    for (const page of frictionlessPdfPages) {
      const url = buildUrl(env, localeInfo.locale, page);
      const result = await checkEdgeWorker(url);
      
      totalChecked++;
      
      if (result.success) {
        if (result.isOnEdgeWorker) {
          onEdgeWorker++;
          console.log(`  ✅ ${page}: ON EdgeWorker`);
        } else {
          console.log(`  ❌ ${page}: NOT on EdgeWorker`);
        }
      } else {
        errors++;
        console.log(`  ⚠️  ${page}: ERROR - ${result.error}`);
      }
      
      localeResults.push({
        page,
        url,
        ...result
      });
      
      // Add small delay to avoid rate limiting
      await delay(100);
    }
    
    if (localeInfo.locale === 'in') {
      for (const page of compressPdfVariations) {
        const url = buildUrl(env, localeInfo.locale, page);
        const result = await checkEdgeWorker(url);
        
        if (result.success) {
          if (result.isOnEdgeWorker) {
            onEdgeWorker++;
            console.log(`  ✅ ${page}: ON EdgeWorker`);
          } else {
            console.log(`  ❌ ${page}: NOT on EdgeWorker`);
          }
        } else {
          errors++;
          console.log(`  ⚠️  ${page}: ERROR - ${result.error}`);
        }
        
        localeResults.push({
          page,
          url,
          ...result
        });
        
        // Add small delay to avoid rate limiting
        await delay(100);
      }
    }

    results.push({
      locale: localeInfo,
      results: localeResults
    });
  }
  
  
  console.log('\n' + '=' .repeat(80));
  console.log('SUMMARY:');
  console.log(`Total URLs checked: ${totalChecked}`);
  console.log(`On EdgeWorker: ${onEdgeWorker} (${((onEdgeWorker / totalChecked) * 100).toFixed(1)}%)`);
  console.log(`NOT on EdgeWorker: ${totalChecked - onEdgeWorker - errors} (${(((totalChecked - onEdgeWorker - errors) / totalChecked) * 100).toFixed(1)}%)`);
  console.log(`Errors: ${errors} (${((errors / totalChecked) * 100).toFixed(1)}%)`);
  
  return results;
}

// Run the check
const allResults = await checkAllUrls('stage');


Starting EdgeWorker check for all frictionless PDF pages in T1 locales...

Checking US (English) (default)...
  ✅ pdf-to-word: ON EdgeWorker
  ✅ pdf-to-jpg: ON EdgeWorker
  ✅ pdf-to-excel: ON EdgeWorker
  ✅ pdf-to-ppt: ON EdgeWorker
  ✅ convert-pdf: ON EdgeWorker
  ✅ word-to-pdf: ON EdgeWorker
  ✅ jpg-to-pdf: ON EdgeWorker
  ✅ png-to-pdf: ON EdgeWorker
  ✅ excel-to-pdf: ON EdgeWorker
  ✅ ppt-to-pdf: ON EdgeWorker
  ✅ ocr-pdf: ON EdgeWorker
  ✅ compress-pdf: ON EdgeWorker
  ✅ pdf-editor: ON EdgeWorker
  ✅ merge-pdf: ON EdgeWorker
  ✅ split-pdf: ON EdgeWorker
  ✅ crop-pdf: ON EdgeWorker
  ✅ delete-pdf-pages: ON EdgeWorker
  ✅ rotate-pdf: ON EdgeWorker
  ✅ rearrange-pdf: ON EdgeWorker
  ✅ extract-pdf-pages: ON EdgeWorker
  ✅ add-pages-to-pdf: ON EdgeWorker
  ✅ add-pdf-page-numbers: ON EdgeWorker
  ✅ sign-pdf: ON EdgeWorker
  ✅ request-signature: ON EdgeWorker
  ✅ password-protect-pdf: ON EdgeWorker
  ✅ ai-chat-pdf: ON EdgeWorker

Checking Germany (de)...
  ✅ pdf-to-word: ON EdgeWorker
  ✅

In [13]:
// Detailed analysis by locale
console.log('\n' + '=' .repeat(80));
console.log('DETAILED ANALYSIS BY LOCALE:');
console.log('=' .repeat(80));

for (const localeData of allResults) {
  const { locale, results } = localeData;
  const onEdgeWorker = results.filter(r => r.isOnEdgeWorker).length;
  const errors = results.filter(r => !r.success).length;
  const total = results.length;
  
  console.log(`\n${locale.name} (${locale.locale || 'default'}):`);
  console.log(`  Total pages: ${total}`);
  console.log(`  On EdgeWorker: ${onEdgeWorker} (${((onEdgeWorker / total) * 100).toFixed(1)}%)`);
  console.log(`  NOT on EdgeWorker: ${total - onEdgeWorker - errors} (${(((total - onEdgeWorker - errors) / total) * 100).toFixed(1)}%)`);
  console.log(`  Errors: ${errors} (${((errors / total) * 100).toFixed(1)}%)`);
  
  // Show pages NOT on EdgeWorker
  const notOnEdgeWorker = results.filter(r => r.success && !r.isOnEdgeWorker);
  if (notOnEdgeWorker.length > 0) {
    console.log('  Pages NOT on EdgeWorker:');
    for (const result of notOnEdgeWorker) {
      console.log(`    - ${result.page}`);
    }
  }
  
  // Show errors
  const errorResults = results.filter(r => !r.success);
  if (errorResults.length > 0) {
    console.log('  Errors:');
    for (const result of errorResults) {
      console.log(`    - ${result.page}: ${result.error}`);
    }
  }
}



DETAILED ANALYSIS BY LOCALE:

US (English) (default):
  Total pages: 26
  On EdgeWorker: 26 (100.0%)
  NOT on EdgeWorker: 0 (0.0%)
  Errors: 0 (0.0%)

Germany (de):
  Total pages: 26
  On EdgeWorker: 26 (100.0%)
  NOT on EdgeWorker: 0 (0.0%)
  Errors: 0 (0.0%)

France (fr):
  Total pages: 26
  On EdgeWorker: 26 (100.0%)
  NOT on EdgeWorker: 0 (0.0%)
  Errors: 0 (0.0%)

Japan (jp):
  Total pages: 26
  On EdgeWorker: 26 (100.0%)
  NOT on EdgeWorker: 0 (0.0%)
  Errors: 0 (0.0%)

United Kingdom (uk):
  Total pages: 26
  On EdgeWorker: 26 (100.0%)
  NOT on EdgeWorker: 0 (0.0%)
  Errors: 0 (0.0%)

Spain (es):
  Total pages: 26
  On EdgeWorker: 26 (100.0%)
  NOT on EdgeWorker: 0 (0.0%)
  Errors: 0 (0.0%)

Italy (it):
  Total pages: 26
  On EdgeWorker: 26 (100.0%)
  NOT on EdgeWorker: 0 (0.0%)
  Errors: 0 (0.0%)

India (in):
  Total pages: 32
  On EdgeWorker: 32 (100.0%)
  NOT on EdgeWorker: 0 (0.0%)
  Errors: 0 (0.0%)

Brazil (br):
  Total pages: 26
  On EdgeWorker: 26 (100.0%)
  NOT on Edge

In [14]:
// Analysis by page type
console.log('\n' + '=' .repeat(80));
console.log('ANALYSIS BY PAGE TYPE:');
console.log('=' .repeat(80));

// Group results by page
const pageAnalysis = {};

for (const page of frictionlessPdfPages) {
  pageAnalysis[page] = {
    total: 0,
    onEdgeWorker: 0,
    errors: 0,
    localeResults: []
  };
  
  for (const localeData of allResults) {
    const result = localeData.results.find(r => r.page === page);
    if (result) {
      pageAnalysis[page].total++;
      if (result.success) {
        if (result.isOnEdgeWorker) {
          pageAnalysis[page].onEdgeWorker++;
        }
      } else {
        pageAnalysis[page].errors++;
      }
      pageAnalysis[page].localeResults.push({
        locale: localeData.locale,
        ...result
      });
    }
  }
}

// Sort pages by EdgeWorker coverage
const sortedPages = Object.entries(pageAnalysis)
  .map(([page, data]) => ({
    page,
    ...data,
    coverage: data.total > 0 ? (data.onEdgeWorker / data.total) * 100 : 0
  }))
  .sort((a, b) => b.coverage - a.coverage);

console.log('\nPages sorted by EdgeWorker coverage:');
for (const pageData of sortedPages) {
  console.log(`${pageData.page}: ${pageData.onEdgeWorker}/${pageData.total} (${pageData.coverage.toFixed(1)}%)`);
  
  // Show which locales are missing EdgeWorker for this page
  const missingLocales = pageData.localeResults.filter(r => r.success && !r.isOnEdgeWorker);
  if (missingLocales.length > 0) {
    console.log(`  Missing in: ${missingLocales.map(r => r.locale.name).join(', ')}`);
  }
}



ANALYSIS BY PAGE TYPE:

Pages sorted by EdgeWorker coverage:
pdf-to-word: 9/9 (100.0%)
pdf-to-jpg: 9/9 (100.0%)
pdf-to-excel: 9/9 (100.0%)
pdf-to-ppt: 9/9 (100.0%)
convert-pdf: 9/9 (100.0%)
word-to-pdf: 9/9 (100.0%)
jpg-to-pdf: 9/9 (100.0%)
png-to-pdf: 9/9 (100.0%)
excel-to-pdf: 9/9 (100.0%)
ppt-to-pdf: 9/9 (100.0%)
ocr-pdf: 9/9 (100.0%)
compress-pdf: 9/9 (100.0%)
pdf-editor: 9/9 (100.0%)
merge-pdf: 9/9 (100.0%)
split-pdf: 9/9 (100.0%)
crop-pdf: 9/9 (100.0%)
delete-pdf-pages: 9/9 (100.0%)
rotate-pdf: 9/9 (100.0%)
rearrange-pdf: 9/9 (100.0%)
extract-pdf-pages: 9/9 (100.0%)
add-pages-to-pdf: 9/9 (100.0%)
add-pdf-page-numbers: 9/9 (100.0%)
sign-pdf: 9/9 (100.0%)
request-signature: 9/9 (100.0%)
password-protect-pdf: 9/9 (100.0%)
ai-chat-pdf: 9/9 (100.0%)


In [ ]:
// Save results to a JSON file for further analysis
const reportData = {
  timestamp: new Date().toISOString(),
  summary: {
    totalUrls: allResults.reduce((sum, locale) => sum + locale.results.length, 0),
    onEdgeWorker: allResults.reduce((sum, locale) => 
      sum + locale.results.filter(r => r.isOnEdgeWorker).length, 0),
    errors: allResults.reduce((sum, locale) => 
      sum + locale.results.filter(r => !r.success).length, 0)
  },
  t1Locales,
  frictionlessPdfPages,
  compressPdfVariations,
  results: allResults
};

// Display final summary
console.log('\n' + '=' .repeat(80));
console.log('FINAL REPORT:');
console.log('=' .repeat(80));
console.log(`Report generated: ${reportData.timestamp}`);
console.log(`Total URLs checked: ${reportData.summary.totalUrls}`);
console.log(`On EdgeWorker: ${reportData.summary.onEdgeWorker} (${((reportData.summary.onEdgeWorker / reportData.summary.totalUrls) * 100).toFixed(1)}%)`);
console.log(`NOT on EdgeWorker: ${reportData.summary.totalUrls - reportData.summary.onEdgeWorker - reportData.summary.errors} (${(((reportData.summary.totalUrls - reportData.summary.onEdgeWorker - reportData.summary.errors) / reportData.summary.totalUrls) * 100).toFixed(1)}%)`);
console.log(`Errors: ${reportData.summary.errors} (${((reportData.summary.errors / reportData.summary.totalUrls) * 100).toFixed(1)}%)`);

// Show recommendations
console.log('\nRECOMMENDATIONS:');
if (reportData.summary.onEdgeWorker === reportData.summary.totalUrls - reportData.summary.errors) {
  console.log('✅ All frictionless PDF pages in T1 locales are on EdgeWorker!');
} else {
  console.log('❌ Some frictionless PDF pages in T1 locales are NOT on EdgeWorker.');
  console.log('   Consider enabling EdgeWorker for these pages to improve performance.');
}

if (reportData.summary.errors > 0) {
  console.log('⚠️  Some URLs returned errors. Check if these pages exist or if there are access issues.');
}

console.log('\n' + '=' .repeat(80));
console.log('Report data available in the reportData variable for further analysis.');
